# Fitting to the experimental data

In this notebook we will show how to load a CIF file, an experimental profile and how to perform a parameter fit.


#### Import Python packages

In [1]:
# esyScience, technique-independent
from easyCore import np
from easyCore.Fitting.Fitting import Fitter

# esyScience, diffraction
try:
    from easyDiffractionLib import Phase
except ImportError:
    import sys, os
    sys.path.append(os.path.abspath('..'))
    
from easyDiffractionLib import Phases
from easyDiffractionLib.sample import Sample as Job
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters as CWParams

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

GSAS-II binary directory: /srv/conda/envs/notebook/lib/python3.7/site-packages/GSASII/bindist
ImportError for wx/mpl in GSASIIctrlGUI: ignore if docs build


In [2]:
output_notebook()
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300

Loading BokehJS ...

## Sample

#### Show a CIF file content

In [3]:
cif_fname = 'PbSO4.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

data_PbSO4

_space_group_name_H-M_alt   'P n m a'

_cell_length_a       8.480
_cell_length_b       5.398
_cell_length_c       6.958
_cell_angle_alpha   90.0
_cell_angle_beta    90.0
_cell_angle_gamma   90.0

loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb   0.188   0.25   0.167   1.0   Uiso  0.01
  S   S    0.063   0.25   0.686   1.0   Uiso  0.01
  O1  O   -0.095   0.25   0.600   1.0   Uiso  0.01
  O2  O    0.181   0.25   0.543   1.0   Uiso  0.01
  O3  O    0.085   0.026  0.806   1.0   Uiso  0.01



#### Load structure from a CIF file

In [4]:
phases = Phases.from_cif_file(cif_fname)
phase = phases[0]

print(phases)
print(phase)

Collection of 1 phases.
Phase `PbSO4`


#### Visualise the structure

In [5]:
structure = py3Dmol.view()
structure.addModel(phase.to_cif_str(), 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Experiment

#### Show measured data as text

In [6]:
meas_fname = 'D1A@ILL.xye'

with open(meas_fname, 'r') as f:
    content = f.read()
    
print('\n'.join(content.split('\n')[:11]))

# PbSO4 D1A(ILL)(Rietveld Refinement Round Robin, R.J. Hill, JApC 25, 589 (1992)
       10.0000          220.0000         14.8324
       10.0500          214.0000         14.6287
       10.1000          219.0000         14.7986
       10.1500          224.0000         14.9666
       10.2000          198.0000         14.0712
       10.2500          229.0000         15.1327
       10.3000          224.0000         14.9666
       10.3500          216.0000         14.6969
       10.4000          202.0000         14.2127
       10.4500          229.0000         15.1327


#### Load the measured data

In [7]:
meas_x, meas_y, meas_e = np.loadtxt(meas_fname, unpack=True)

#### Visualize the measured data

In [8]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
show(fig)

## Analysis

#### Create job with default parameters for the 1D powder neutron diffraction experiment with constant wavelength 

In [9]:
calculator = Calculator(interface_name='CrysPy')

In [10]:
print(f"Current calculator engine: {calculator.current_interface_name}")

Current calculator engine: CrysPy


In [11]:
job = Job(phases=phases, parameters=CWParams.default(), calculator=calculator)

Temp CIF: /tmp/easydiffraction_temp.cif


#### Generate the calculated data

In [12]:
calc_y_cryspy = calculator.fit_func(meas_x)

#### Visualize both the measured and calculated data

In [13]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Set scale manually

In [14]:
job.pattern.scale = 100

In [15]:
calc_y_cryspy = calculator.fit_func(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Set wavelength manually

In [16]:
job.parameters.wavelength = 1.912

In [17]:
calc_y_cryspy = calculator.fit_func(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Set background points manually

In [18]:
bkg = PointBackground(linked_experiment='PbSO4')

bkg.append(BackgroundPoint.from_pars(meas_x[0], 200))
bkg.append(BackgroundPoint.from_pars(meas_x[-1], 250))

job.set_background(bkg)

In [19]:
calc_y_cryspy = calculator.fit_func(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Define parameters to optimize

In [20]:
job.pattern.scale.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.resolution_u.fixed = False
job.parameters.resolution_v.fixed = False
job.parameters.resolution_w.fixed = False
job.backgrounds[0][0].y.fixed = False
job.backgrounds[0][1].y.fixed = False

In [21]:
print(job.pattern.scale)
print(job.pattern.zero_shift)
print(job.parameters.resolution_u)
print(job.parameters.resolution_v)
print(job.parameters.resolution_w)
print(job.backgrounds[0][0])
print(job.backgrounds[0][1])

<Parameter 'scale': 100.0+/-0, bounds=[-inf:inf]>
<Parameter 'zero_shift': 0.0+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_u': 0.0002+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.0002+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.012+/-0, bounds=[-inf:inf]>
<BackgroundPoint '10,0_deg': 200.0+/-0, bounds=[-inf:inf]>
<BackgroundPoint '120,0_deg': 250.0+/-0, bounds=[-inf:inf]>


#### Initalize the fitting engine and perform the fit

In [22]:
fitter = Fitter(job, calculator.fit_func)

In [23]:
print(f"Available minimizers: {fitter.available_engines}")
print(f"Current minimizer: {fitter.current_engine.name}")
print(f"Available methods of current minimizers: {fitter.available_methods()}")

Available minimizers: ['lmfit', 'bumps', 'DFO_LS']
Current minimizer: lmfit
Available methods of current minimizers: ['least_squares', 'leastsq', 'differential_evolution', 'basinhopping', 'ampgo', 'nelder', 'lbfgsb', 'powell', 'cg', 'newton', 'cobyla', 'bfgs']


In [24]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, 
                    method='least_squares', minimizer_kwargs={'diff_step': 1e-5})

In [25]:
print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

The fit has been successful: True
The gooodness of fit (chi2) is: 31.613447982842985
<Parameter 'scale': 600.5398381881878+/-0, bounds=[-inf:inf]>
<Parameter 'zero_shift': 0.12019197045437276+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_u': 0.1889656195775758+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.46198067218375155+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.4705846207697826+/-0, bounds=[-inf:inf]>
<BackgroundPoint '10,0_deg': 198.88369792536517+/-0, bounds=[-inf:inf]>
<BackgroundPoint '120,0_deg': 239.3945793842329+/-0, bounds=[-inf:inf]>


In [26]:
calc_y_cryspy = calculator.fit_func(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
fig.line(meas_x, meas_y-calc_y_cryspy, legend_label='Imeas - Icalc (CrysPy)', color='olivedrab', line_width=2)
show(fig)

#### Change calculator engine to CrysFML

In [27]:
print(f"Available calculator engines: {calculator.available_interfaces}")

Available calculator engines: ['CrysPy', 'CrysFML', 'GSASII']


In [28]:
job.interface.switch('CrysFML', fitter=fitter)

In [29]:
print(f"Current calculator engine: {job.interface.current_interface_name}")
print(f"Current minimizer: {fitter.current_engine.name}")

Current calculator engine: CrysFML
Current minimizer: lmfit


#### Show results of both CrysPy and CrysFML calculations (before fitting)

In [30]:
calc_y_crysfml = calculator.fit_func(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
fig.line(meas_x, calc_y_crysfml, legend_label='Icalc (CrysFML)', color='orange', line_width=2)
show(fig)

#### Perform the fit with CrysFML

In [31]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, 
                    method='least_squares', minimizer_kwargs={'diff_step': 1e-5})

In [32]:
print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

The fit has been successful: True
The gooodness of fit (chi2) is: 31.728826801731252
<Parameter 'scale': 382.0415205719351+/-0, bounds=[-inf:inf]>
<Parameter 'zero_shift': 0.1214009397728182+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_u': 0.18659040002332336+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.4568616713691494+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.46886658705535994+/-0, bounds=[-inf:inf]>
<BackgroundPoint '10,0_deg': 198.45343313114492+/-0, bounds=[-inf:inf]>
<BackgroundPoint '120,0_deg': 239.49305215873028+/-0, bounds=[-inf:inf]>


#### Show results of both CrysPy and CrysFML calculations (after fitting)

In [34]:
calc_y_crysfml = calculator.fit_func(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
fig.line(meas_x, calc_y_crysfml, legend_label='Icalc (CrysFML)', color='orange', line_width=2)
fig.line(meas_x, calc_y_cryspy-calc_y_crysfml, legend_label='Icalc (CrysPy) - Icalc (CrysFML)', color='grey', line_width=2)
show(fig)

#### Show the difference between CrysPy and CrysFML in calculated patterns

In [35]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, calc_y_cryspy-calc_y_crysfml, legend_label='Icalc (CrysPy) - Icalc (CrysFML)', color='grey', line_width=2)
show(fig)